<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from utils.config import *
local_data_folder = get_local_data_folder('hgdp')
local_data_folder

'C:/Data/HUJI/hgdp/'

In [2]:
# todo - think about the infra to get to each file, and naming conventions

# Process 012 files

In [3]:
#imports and consts
import os.path
import logging
import pandas as pd
import math

SUFFIX_POS = '.012.pos'
SUFFIX_012 = '.012'
OUTPUT_PATTERN_DIST_FILE = 'dist_slice_{slice_index}_{num_of_loci}_loci.tsv'
OUTPUT_PATTERN_LOCI_FILE = 'loci_ids_slice_{slice_index}.tsv'

logger = logging.getLogger(__name__)

# TODO?
#fhandler = logging.FileHandler(filename='Process012.log', mode='a')
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
#fhandler.setFormatter(formatter)
#logger.addHandler(fhandler)

def _log(msg, level=logging.INFO):
    print(msg)
    logger.log(level, msg)

In [4]:
# func params
path_to_012_files = 'C:\Data\HUJI\hgdp\mock\out'
base_output_path = 'C:\Data\HUJI\hgdp\mock'
# if not in range - will raise a warning
min_minor_freq_expected = 0.49
max_minor_freq_expected = 0.5
# if we have 91 loci, we may have a file with 1 loci in it (can be improved if we want..)
# TODO - validate with Gili
desired_num_of_loci_per_output = 10
# TODO - validate with Gili
# if we have less than this which are valied (not -1), locus is not included in calc. This will change the 
min_valid_sites_precentage = 0.1

In [5]:
def _validate_file_exist(p):
    if not os.path.isfile(p):
        _log(f'Cant file file {p}', logging.ERROR)

In [6]:
def _get_chr_name(path_pos):
    with open(path_pos, 'r') as f:
        l = f.readline()
        char_name = l.split('\t')[0]
        return char_name

In [7]:
def _prepare_output_folder(base_output_path, chr_name):
    output_dir = f'{base_output_path}/{chr_name}/'
    _log(f'output_dir is {output_dir}')
    os.makedirs(output_dir, exist_ok=True)
    return output_dir


In [67]:
def _build_pairwise_db(n, v):
    return [[v]*(n-i) for i in range (1,n)]

In [73]:
#pairwise_distances = _calc_pairwise_distances_with_guardrails(slice_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)
# for each column, we calc the pairwise distances and add it to the grand total
# for performance, we use 2 lists of lists, one for distances and one for counts
# maybe wrap in np to sum?
slice_pairwise_dist = _build_pairwise_db(len(slice_df), 0.0)
slice_pairwise_counts = _build_pairwise_db(len(slice_df), 0)

In [74]:
# def _calc_pairwise_distances_with_guardrails_locus
site_index = 0
genotypes = slice_df.iloc[:,site_index].values
# get counts
num_individuals = len(genotypes)
# count the amount of not -1 in alleles
num_valid_genotypes = len(genotypes[genotypes!=-1])
non_ref_count = sum(genotypes[genotypes>0])
ref_count = 2*num_valid_genotypes-non_ref_count
non_ref_freq = float(non_ref_count)/(2*num_valid_genotypes)
ref_freq = float(ref_count)/(2*num_valid_genotypes)
_log(f'Non ref allele frequency: {non_ref_freq}')
_log(f'Ref allele frequency: {ref_freq}')
assert abs(ref_freq+non_ref_freq-1)<1e-04
_check_guardrails(num_individuals, num_valid_genotypes, ref_count, non_ref_count, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)

Non ref allele frequency: 0.5
Ref allele frequency: 0.5
Check guardrails
Precentage of valid sites: 0.4951560818083961
Minor allele frequency: 0.5
Passed guardrails


In [75]:
def test(v1_val, v2_val):
    print(v1_val, v2_val, f'(1-f_a){(v1_val*v2_val)} + (1-f_r){(2-v1_val)*(2-v2_val)}')
for v1 in range(3):
    for v2 in range(v1,3):
        test(v1, v2)

0 0 (1-f_a)0 + (1-f_r)4
0 1 (1-f_a)0 + (1-f_r)2
0 2 (1-f_a)0 + (1-f_r)0
1 1 (1-f_a)1 + (1-f_r)1
1 2 (1-f_a)2 + (1-f_r)0
2 2 (1-f_a)4 + (1-f_r)0


In [76]:
def _calc_dist(i1_val, i2_val, ref_freq, non_ref_freq):
    # from VCFtools manual:
    # "Genotypes are represented as 0, 1 and 2, where the number represent that number of non-reference alleles"
    # So - v1_val and v2_val are the amount of non-ref alleles.
    # The distance function is:
    # 1/4[(1-f_a)(Iac+Iad) + (1-f_b)(Ibc+Ibd)]
    # Now, for each combination of v1_val and v2_val, we can compute the distance.
        
    # 0,0 - v1_val=0 and v2_val=0:
    #    1/4[(1-f_ref)(1+1) + (1-f_ref)(1+1)] = 1/4[(1-f_ref)(4)] = (1-f_ref)
    
    # 0,1 - v1_val=0 and v2_val=1:
    #    1/4[(1-f_ref)(1+0) + (1-f_ref)(1+0)] = 1/4[(1-f_ref)(2)] = 1/2(1-f_ref)
    
    # 0,2 - v1_val=0 and v2_val=2:
    #    1/4[(1-f_ref)(0+0) + (1-f_ref)(0+0)] = 0
    
    # 1,1 - v1_val=1 and v2_val=2:
    #    1/4[(1-f_non_ref)(1+0) + (1-f_ref)(1+0)] = 1/4[(1-f_non_ref)+(1-f_ref)]
    
    # 1,2 - v1_val=1 and v2_val=2:
    #    1/4[(1-f_non_ref)(1+1) + (1-f_ref)(0+0)] = 1/4[(1-f_non_ref)(2)] = 1/2(1-f_non_ref)
    
    # 2,2 - v1_val=2 and v2_val=2:
    #    1/4[(1-f_non_ref)(1+1) + (1-f_non_ref)(1+1)] = 1/4[(1-f_non_ref)(4)] = (1-f_non_ref)
    
    # Also, note that it is symetric:

    # 1,0 - v1_val=1 and v2_val=0:
    #    1/4[(1-f_ref)(1+1) + (1-f_non_ref)(0+0)] = 1/4[(1-f_ref)(2)] = 1/2(1-f_ref)
    
    # 2,1 - v1_val=2 and v2_val=1:
    #    1/4[(1-f_non_ref)(1+0) + (1-f_non_ref)(0+1)] = 1/4[(1-f_non_ref)(2)] = 1/2(1-f_non_ref)

    # 2,0 - v1_val=2 and v2_val=0:
    #    1/4[(1-f_non_ref)(0+0) + (1-f_non_ref)(0+0)] = 0
    
    # formula to use: 
    #   (1-f_non_ref)(v1_val*v2_val) + (1-f_ref)((2-v1_val)*(2-v2_val))/4
    return (1-non_ref_freq)*(i1_val*i2_val) + (1-ref_freq)*((2-i1_val)*(2-i2_val))/4
    

In [82]:
sec = 0.5
10000*sec/(60)

83.33333333333333

In [83]:
# now when we have all freq and counts, we can start the pairwise comparison
# takes 0.5 seconds for one locus. This means ~4 days if we are not seperating the task (which we will)
# Note that if we split this by chromozome, the big ones may have around 1000 in a given freq range, 
# which can take ~8 minutes. 10k will be around 83 minutes - also cool.

for i1 in range(num_individuals-1):
    i1_val = allels[i1]
    # if this entry is not valid for i1, no need to go over all the others, nothing to add to freq nor counts
    if i1%100==0:
        _log(f'Site index: {site_index}: done with individual {i1}')
    if i1_val == -1:
        continue
    for i2 in range(i1+1, num_individuals):
        i2_val = allels[i2]
        if i2_val == -1:
            continue
        else:            
            # this is a valid entry, we add 1 to the count
            slice_pairwise_counts[i1][i2-i1-1] += 1
            slice_pairwise_dist[i1][i2-i1-1] += _calc_dist(i1_val, i2_val, ref_freq, non_ref_freq)


Site index: 0: done with individual 0
Site index: 0: done with individual 100
Site index: 0: done with individual 200
Site index: 0: done with individual 300
Site index: 0: done with individual 400
Site index: 0: done with individual 500
Site index: 0: done with individual 600
Site index: 0: done with individual 700
Site index: 0: done with individual 800
Site index: 0: done with individual 900


In [72]:
slice_pairwise_dist[-10:]

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 2.0, 2.0],
 [0.0, 0.0, 0.0],
 [0.0, 0.0],
 [2.0]]

In [9]:
def _check_guardrails(num_individuals, num_valid_genotypes, ref_count, non_ref_count, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected):
    _log(f'Check guardrails')
    # guardrail #1 - min_valid_sites_precentage
    percentage_valid_sites = float(num_valid_genotypes)/num_individuals
    _log(f'Precentage of valid sites: {percentage_valid_sites}')
    if percentage_valid_sites < min_valid_sites_precentage:
        _log(f'ERROR: % of valid sites is {percentage_valid_sites}, lower than allowd: {min_valid_sites_precentage}.', logging.ERROR)
    # guardrail #2 - min_minor_freq_expected and max_minor_freq_expected
    minor_count = min(ref_count, non_ref_count)
    minor_freq = float(minor_count)/(2*num_valid_genotypes)
    _log(f'Minor allele frequency: {minor_freq}')
    if minor_freq < min_minor_freq_expected:
        _log(f'ERROR: minor frequency is too low - {minor_freq}, allowd: {min_minor_freq_expected}.', logging.ERROR)
    if minor_freq > max_minor_freq_expected:
        _log(f'ERROR: minor frequency is too high - {minor_freq}, allowd: {max_minor_freq_expected}.', logging.ERROR)
    _log(f'Passed guardrails')
    

In [82]:
# def _calc_pairwise_distances_with_guardrails
locus_pairwise_dist, locus_pairwise_counts =  _calc_pairwise_distances_with_guardrails_locus(allels, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)
# add to slice

In [10]:
def _write_output_loci_file(output_loci_file, slice_df):
    loci_ids = slice_df.columns.values
    with open(output_loci_file, 'w') as f:
        for locus_id in loci_ids:
            f.write(locus_id.replace('_','\t') + '\n')


# Func begin

In [11]:
# outputs:
# for each group i in 1,...,k:
# 1. {i}_dist_{l}_loci : tab seperated pairwise distances - this is the sum over the valid loci used, and the format in each entry is <sum>,<l'>, or <sum> if l'=l (from the file's name)
# the idea here is that in most cases all entries are valid for both individuals, and there is no need to store this value. In the cases there are l'>l valid sites, we add l' to the output in the entry.
# dummy example for 3 individuals, over l=5, where individual number 3 has 2 invalid entries (-1 -1 1 0 2)
# 0.9 0.7,3
# 0.8,3 
# the ',3' indicates that there are only 3 valied sited for individuals 1 and 3 and 2 and 3
# 2. {i}_loci_ids : <locus_id> <number of non -1 entris>

# Validate inputs
path_012 = f'{path_to_012_files}{SUFFIX_012}'
path_pos = f'{path_to_012_files}{SUFFIX_POS}'
_validate_file_exist(path_012)
_validate_file_exist(path_pos)

# prepare output folder
chr_name = _get_chr_name(path_pos)
_log(f'chr_name is {chr_name}')
output_dir = _prepare_output_folder(base_output_path, chr_name)

# load all data

chr_name is chr9
output_dir is C:\Data\HUJI\hgdp\mock/chr9/


In [12]:
# get loci positions to use as column names
with open(path_pos, 'r') as f:
    loci_pos = [l.replace('\t','_').replace('\n','') for l in f.readlines()]
all_df = pd.read_csv(path_012, sep='\t', names = ['ind_id'] + loci_pos)
# drop first column - individual ids
all_df = all_df.drop(columns=['ind_id'])

In [13]:
num_loci = len(all_df.columns)
num_of_slices = math.ceil(float(num_loci)/desired_num_of_loci_per_output)
size_of_last_slice = num_loci % desired_num_of_loci_per_output
_log(f'Total num of loci: {num_loci}')
_log(f'Desired size of slice: {desired_num_of_loci_per_output}')
_log(f'Number of slices: {num_of_slices}')
_log(f'Size of last slice: {size_of_last_slice}')
for slice_index in  range(num_of_slices):
    min_col_index = slice_index*desired_num_of_loci_per_output
    # in case we have less than desired_num_of_loci_per_output, pandas will only take those we have!
    max_col_index = min_col_index + desired_num_of_loci_per_output
    slice_df = all_df.iloc[:,min_col_index:max_col_index]
    num_of_loci = len(slice_df.columns)
    _log(f'\n#############\n')
    _log(f'Slice index: {slice_index}. Slice size: {num_of_loci} ')
    output_dist_file = output_dir + OUTPUT_PATTERN_DIST_FILE.format(slice_index=slice_index, num_of_loci=num_of_loci)
    output_loci_file = output_dir + OUTPUT_PATTERN_LOCI_FILE.format(slice_index=slice_index)
    # todo - maybe write to this file also the number of -1, 0, 1 and 2?
    _write_output_loci_file(output_loci_file, slice_df)
    _log(f'Output files: {output_dist_file} {output_loci_file}')
    pairwise_distances = _calc_pairwise_distances_with_guardrails(slice_df, min_valid_sites_precentage, min_minor_freq_expected, max_minor_freq_expected)
    _write_pairwise_distances(output_dist_file, pairwise_distances)

Total num of loci: 178
Desired size of slice: 10
Number of slices: 18
Size of last slice: 8

#############

Slice index: 0. Slice size: 10 
Output files: C:\Data\HUJI\hgdp\mock/chr9/dist_slice_0_10_loci.tsv C:\Data\HUJI\hgdp\mock/chr9/loci_ids_slice_0.tsv


NameError: name '_calc_pairwise_distances_with_guardrails' is not defined